In [ ]:
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from ipywidgets import HBox, VBox, HTML
from IPython.display import display
from giga.app.config import get_registered_country_names
from giga.viz.notebooks.components.html.sections import section
from giga.data.space.country_updater import CountryUpdateRequest, CountryUpdater
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.schemas.conf.country import CountryDefaults
from giga.data.store.stores import COUNTRY_DATA_STORE as data_store
from giga.utils.globals import *
import pandas as pd
import json
from traitlets import link, Tuple, Unicode
from typing import Any
from giga.viz.notebooks.parameters.groups.data_parameter_manager import country_name_to_key

def update_selected_country_fiber_cache(ev):
    with update_cache_output:
        CountryUpdater.update_fiber_cache(get_selected_country())

def update_selected_country_cellular_cache(ev):
    with update_cache_output:
        CountryUpdater.update_cellular_cache(get_selected_country())

def update_selected_country_p2p_cache(ev):
    with update_cache_output:
        CountryUpdater.update_p2p_cache(get_selected_country())
        
def update_selected_country_schools_cache(ev):
    with update_cache_output:
        CountryUpdater.update_schools_cache(get_selected_country())
        
def update_selected_country_schools_visibility_cache(ev):
    with update_cache_output:
        CountryUpdater.update_schools_visibility_cache(get_selected_country())


country_select = widgets.Dropdown(
    options=["None"] + get_registered_country_names(),
    description='Country:',
    value = "None",
)


update_fiber_cache_btn = widgets.Button(description="Update fiber cache")
update_fiber_cache_btn.on_click(update_selected_country_fiber_cache)
update_cellular_cache_btn = widgets.Button(description="Update cellular cache")
update_cellular_cache_btn.on_click(update_selected_country_cellular_cache)
update_p2p_cache_btn = widgets.Button(description="Update p2p cache")
update_p2p_cache_btn.on_click(update_selected_country_p2p_cache)
update_schools_cache_btn = widgets.Button(description="Update schools cache")
update_schools_cache_btn.on_click(update_selected_country_schools_cache)
update_schools_visibility_cache_btn = widgets.Button(description="Update schools visibility cache")
update_schools_visibility_cache_btn.on_click(update_selected_country_schools_visibility_cache)

# Define outputs

# TODO use country_name_to_key
output = widgets.Output()
#country_config_output = widgets.Output()
optional_info_output = widgets.Output()
save_output = widgets.Output()
full_config_output = widgets.Output()
fetching_data_output = widgets.Output()
#delete_btn_output = widgets.Output()
update_cache_output = widgets.Output()
update_fiber_cache_output = widgets.Output()
update_cellular_cache_output = widgets.Output()
update_p2p_cache_output = widgets.Output()

# Event handlers

first_time = True
def on_selected_country_changed(change):
    # Show the appropriate country widget depending on operation.
    output.clear_output()
    with output:
        fetch_data()
    update_cache_output.clear_output()
    show_update_fiber_cache_btn()
    show_update_cellular_cache_btn()
    show_update_p2p_cache_btn()
    show_update_schools_cache_btn()
    show_update_schools_visibility_cache_btn()


def show_update_fiber_cache_btn():
    with update_cache_output:
        display(update_fiber_cache_btn)

def show_update_cellular_cache_btn():
    with update_cache_output:
        display(update_cellular_cache_btn)

def show_update_p2p_cache_btn():
    with update_cache_output:
        display(update_p2p_cache_btn)
        
def show_update_schools_cache_btn():
    with update_cache_output:
        display(update_schools_cache_btn)
        
def show_update_schools_visibility_cache_btn():
    with update_cache_output:
        display(update_schools_visibility_cache_btn)

def fetch_data(workspace="workspace",force_country: str = None):
    global first_time
    fetching_data_output.clear_output()
    country = get_selected_country() if force_country is None else None
    if country!=None:
        country_dir = os.path.join(workspace,country)
        item_list = []
        
        #fiber file
        try:
            with data_store.open(os.path.join(country_dir,FIBER_FILE)) as f:
                df = pd.read_csv(f)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame()
            
        if df.empty:
            item_list.append('No fiber file - fiber cache cannot be calculated - fiber disallowed unless distance to fiber node in schools file')
            #schools cache
            with data_store.open(os.path.join(country_dir,SCHOOLS_CACHE_FILE)) as f:
                js = json.load(f)
                
            if len(js['lookup'])==0:
                item_list.append('No school cache file - fiber disallowed')
        else:
            #schools cache
            with data_store.open(os.path.join(country_dir,SCHOOLS_CACHE_FILE)) as f:
                js = json.load(f)
                
            if len(js['lookup'])==0:
                item_list.append('No school cache file - fiber disallowed')

            #fiber cache
            with data_store.open(os.path.join(country_dir,FIBER_CACHE_FILE)) as f:
                js = json.load(f)
                
            if len(js['lookup'])==0:
                item_list.append('No fiber cache file - fiber disallowed')

        #cell file
        try:
            with data_store.open(os.path.join(country_dir,CELL_FILE)) as f:
                df = pd.read_csv(f)
        except pd.errors.EmptyDataError:
            df = pd.DataFrame()
            
        if df.empty:
            item_list.append('No cellular file - cellular cache cannot be calculated - cellular only if coverage and p2p disallowed')
        else:
            #cell cache
            with data_store.open(os.path.join(country_dir,CELL_CACHE_FILE)) as f:
                js = json.load(f)
                
            if len(js['lookup'])==0:
                item_list.append('No cellular cache file - cellular disallowed') 

            #p2p cache
            with data_store.open(os.path.join(country_dir,P2P_CACHE_FILE)) as f:
                js = json.load(f)
                
            if len(js['lookup'])==0:
                item_list.append('No p2p cache file - p2p disallowed')

        html_content = "<ul>" + "".join(f"<li>{item}</li>" for item in item_list) + "</ul>"
        with fetching_data_output:
            display(section(
            title="",
            contents=HTML(html_content)
            ))
        
    
    if not first_time:
        return
    first_time = False
    with full_config_output:
    #    display(country_config_output)
        display(optional_info_output)
        display(save_output)


# Update the page when the selected country changes
country_select.observe(on_selected_country_changed, 'value')

# Display output

display(section(
    title="Update Countries",
    contents=VBox([
        HTML("See <a href=\"notebooks/dev/update-objstore.ipynb\">documentation</a>"),
        country_select,
        output
    ]),
    extra_class="header"))
display(fetching_data_output,full_config_output)

# Bind callbacks



def get_selected_country():
    
    if country_select.value == "None":
        return None
    return country_name_to_key(country_select.value)
    #return None

In [ ]:

def up_btn(description, accept=".csv", **kwargs):
    layout = widgets.Layout(width='300px')
    return widgets.FileUpload(accept=accept, multiple=False, 
                              description=description, layout=layout, 
                              max_file_size=100,  # MB
                              **kwargs)

class FileUploadButton(widgets.HBox):
    value = Tuple().tag(sync=True)
    description = Unicode().tag(sync=True)
    def __init__(self, description, accept=".csv", **kwargs):
        self.upload_button = up_btn(accept=accept, description=description, **kwargs)
        self.clear_button = widgets.Button(icon='times', layout=dict(width='auto'))

        super().__init__([self.upload_button, self.clear_button])

        self.clear_button.on_click(self.clear_upload)

        # Link values to be read directly from our fake upload button
        self.value = self.upload_button.value
        link((self.upload_button, 'value'), (self, 'value'))
        self.description = self.upload_button.description
        link((self.upload_button, 'description'), (self, 'description'))

    def clear_upload(self, event):
        self.upload_button.value = ()

# Now use this class to create the optional file upload buttons.
cellular = FileUploadButton(description="Cell tower locations (.csv)")
fiber = FileUploadButton(description="Fiber node locations (.csv)")
#schools_supplemental = FileUploadButton(description="Supplemental school info (.csv)")

upload_output = widgets.Output()
upload_button = widgets.Button(description="Save Country Files")

import ipywidgets as widgets
from traitlets import observe, link, Unicode, Bool, Any




with optional_info_output:
    display(section(
        title="Optional information",
        contents=VBox([
            cellular,
            fiber,
            #schools_supplemental,
        ]),
        extra_class="center"
    ))

with save_output:
    display(section(
        title="Save",
        contents=VBox([
            upload_button,
            upload_output,
            HTML('<br/><br/>'),
            HBox([
                update_cache_output
                #delete_btn_output
            ])
        ]),
        extra_class="footer"
    ))

def on_upload_change(change):
    upload_output.clear_output()
    with upload_output:
        # Create a new update request
        req = CountryUpdateRequest()
        req.country_key = get_selected_country(),
        req.cellular = cellular
        req.fiber = fiber

        # Update ObjStore
        if req.attempt():
            display(HTML("Successfully updated! Please update the cache before running the app on this country."))
        

upload_button.on_click(on_upload_change)